In [1]:
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas
from argparse import ArgumentParser

import lightning.pytorch as pl
from lightning.pytorch import LightningModule, Trainer
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
from lightning.pytorch import LightningModule
from lightning.pytorch.loggers import TensorBoardLogger
from sklearn.preprocessing import MinMaxScaler
from deep_traffic_generation.fcvae import FCVAE

from deep_traffic_generation.core.abstract import AE, VAE
from deep_traffic_generation.core.datasets import TrafficDataset
from deep_traffic_generation.core.lsr import VampPriorLSR, NormalLSR, ExemplarLSR
from deep_traffic_generation.core.networks import FCN, RNN, TCN
from deep_traffic_generation.core.utils import get_dataloaders

# seed
pl.seed_everything(42, workers=True)

# enable autoreload
%load_ext autoreload    
%autoreload 2

Global seed set to 42


In [2]:
# ------------
# args
# ------------
cls = FCVAE
dataset_cls = TrafficDataset
data_shape = "linear"

parser = ArgumentParser()
parser.add_argument(
    "--train_ratio", dest="train_ratio", type=float, default=0.8
)
parser.add_argument(
    "--val_ratio", dest="val_ratio", type=float, default=0.2
)
parser.add_argument(
    "--batch_size", dest="batch_size", type=int, default=1000
)
parser.add_argument(
    "--test_batch_size",
    dest="test_batch_size",
    type=int,
    default=None,
)
parser.add_argument(
    "--early_stop", dest="early_stop", type=int, default=None
)
parser.add_argument(
    "--gradient_clip_val", dest="gradient_clip_val", type=float, default=0
)
parser = dataset_cls.add_argparse_args(parser)
parser, _ = cls.add_model_specific_args(parser)
args = parser.parse_args([])

# Update the args object with the provided values
args.data_path = "../data/toy_dataset.parquet"
args.h_dims = [64, 32, 8]
args.encoding_dim = 8
args.lr_step_size = 200
args.lr = 0.001
args.lrgamma = 0.5
args.dropout = 0.2
args.features = ["track", "groundspeed", "altitude", "timedelta"]
args.info_features = ["latitude", "longitude"]
args.info_index = -1
args.kld_coef = 2



In [3]:
# ------------
# data
# ------------
dataset = dataset_cls.from_file(
    args.data_path,
    features=args.features,
    shape=data_shape,
    scaler=MinMaxScaler(feature_range=(-1, 1)),
    info_params={"features": args.info_features, "index": args.info_index},
)

train_loader, val_loader, test_loader = get_dataloaders(
    dataset,
    args.train_ratio,
    args.val_ratio,
    args.batch_size,
    args.test_batch_size,
)

In [4]:
# ------------
# logger
# ------------
tb_logger = TensorBoardLogger(
    "lightning_logs/",
    name=args.network_name,
    default_hp_metric=False,
    log_graph=True,
)

# ------------
# model
# ------------
model = cls(
    dataset_params=dataset.parameters,
    config=args,
)

In [5]:
# ------------
# training
# ------------
checkpoint_callback = ModelCheckpoint(monitor="hp/valid_loss")
# checkpoint_callback = ModelCheckpoint()
if args.early_stop is not None:
    early_stopping = EarlyStopping(
        "hp/valid_loss", patience=args.early_stop
    )
    trainer = Trainer(
        gradient_clip_val=args.gradient_clip_val,
        callbacks=[checkpoint_callback, early_stopping],
        logger=tb_logger,
        # deterministic=True,
    )
else:
    trainer = Trainer(
        gradient_clip_val=args.gradient_clip_val,
        callbacks=[checkpoint_callback],
        logger=tb_logger,
        # deterministic=True,
    )

if val_loader is not None:
    trainer.fit(model, train_loader, val_loader)
else:
    trainer.fit(model, train_loader)

# ------------
# testing
# ------------
if test_loader is not None:
    trainer.test(test_dataloaders=test_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type      | Params | In sizes | Out sizes
---------------------------------------------------------------
0 | encoder   | FCN       | 53.8 K | [1, 800] | [1, 8]   
1 | lsr       | NormalLSR | 160    | [1, 8]   | ?        
2 | decoder   | FCN       | 54.7 K | [1, 8]   | [1, 800] 
3 | out_activ | Identity  | 0      | [1, 800] | [1, 800] 
---------------------------------------------------------------
108 K     Trainable params
16        Non-trainable params
108 K     Total params
0.435     Total estimated model params size (MB)
c:\Users\Zak\mambaforge\envs\traffic\lib\site-packages\torch\jit\_trace.py:976: TracerWarning: Encountering a list at the output of the tracer might cause the trace to be incorrect, this is only valid if the container structure does not change based on the 

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

TypeError: Trainer.test() got an unexpected keyword argument 'test_dataloaders'